### Movie Recommendation System

<h4>Problem statement:</h4> From the given data recommend 5 movies from one selected movie.

Steps followed:<br>
1. Data Preprocessing
2. Feature selection
3. Model selection
4. Recommendations
5. Deployment using Streamlit


In [2]:
# Regular EDA (exploratory data analysis) and plotting libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
plt.style.use('seaborn')
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

or simplly donwload these datasets from the kaggle directory
https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata?resource=download&select=tmdb_5000_movies.csv
https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata?resource=download&select=tmdb_5000_credits.csv

In [3]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [4]:
movies= movies.merge(credits,on='title')

In [5]:
#features important 
#gernes
#id
#Keywords
#titles
#overview
#casting
#crew

In [6]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [7]:
movies.isnull().sum()
movies.dropna(inplace=True)

In [8]:
movies.duplicated().sum()

0

In [9]:
import ast

In [10]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [11]:
movies['genres'] = movies['genres'].apply(convert)

In [12]:
movies['keywords']=movies['keywords'].apply(convert)

In [13]:
def convert(obj):
    L=[]
    counter  = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter = counter + 1
        else:
            break
    return L

In [14]:
movies['cast'] = movies['cast'].apply(convert)

In [15]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 
    
movies['crew'] = movies['crew'].apply(fetch_director)

In [16]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
2885,8669,Charlie Bartlett,"[Awkward, teenager, Charlie, Bartlett, (Anton,...","[Comedy, Drama]","[depression, suicide attempt, medicine, new lo...","[Anton Yelchin, Robert Downey Jr., Hope Davis]",[Jon Poll]
83,79698,The Lovers,"[The, Lovers, is, an, epic, romance, time, tra...","[Action, Adventure, Science Fiction, Romance]",[],"[Josh Hartnett, Simone Kessell, Tamsin Egerton]",[Roland Joffé]
2819,67660,Think Like a Man,"[The, balance, of, power, in, four, couples’, ...","[Comedy, Romance]","[advice, love, relationship, ensemble cast, du...","[Michael Ealy, Jerry Ferrara, Meagan Good]",[Tim Story]
1135,49730,Red Riding Hood,"[Valerie, is, in, love, with, a, brooding, out...","[Fantasy, Thriller, Horror]","[winter, fantasy, fairy tale, hood, werewolf, ...","[Amanda Seyfried, Gary Oldman, Billy Burke]",[Catherine Hardwicke]
2984,1588,Bring It On,"[The, Toro, cheerleading, squad, from, Rancho,...",[Comedy],"[cheerleader, sport, high school, teenage girl...","[Kirsten Dunst, Jesse Bradford, Eliza Dushku]",[Peyton Reed]


In [17]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [18]:
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

In [19]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


Creating Tags by merging overview , genres , keywords etc.

In [20]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']


Create new dataframe by removing others and just keeping title and tags 

In [21]:
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
#new.head()

In [22]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


Converting each movie to a vector in 5000 Deminsion and then finding the cosine distance between two movies in order to get the nearest movie .
The ones with the least cosine distance are considered to be nearest to each other and hence recommended.

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [24]:
vector = cv.fit_transform(new['tags']).toarray()

AttributeError: 'CountVectorizer' object has no attribute 'accuracy'

In [ ]:
vector.shape

(4806, 5000)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

array([[1.        , 0.08964215, 0.06071767, ..., 0.02519763, 0.0277885 ,
        0.        ],
       [0.08964215, 1.        , 0.06350006, ..., 0.02635231, 0.        ,
        0.        ],
       [0.06071767, 0.06350006, 1.        , ..., 0.02677398, 0.        ,
        0.        ],
       ...,
       [0.02519763, 0.02635231, 0.02677398, ..., 1.        , 0.07352146,
        0.04774099],
       [0.0277885 , 0.        , 0.        , ..., 0.07352146, 1.        ,
        0.05264981],
       [0.        , 0.        , 0.        , ..., 0.04774099, 0.05264981,
        1.        ]])

In [ ]:
new[new['title'] == 'The Lego Movie'].index[0]

744

In [ ]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [ ]:
recommend('Gandhi')

Gandhi, My Father
The Wind That Shakes the Barley
A Passage to India
Guiana 1838
Ramanujan


In [ ]:
import pickle

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))

pickle.dump(new,open('movie_list.pkl','wb'))
